# Table of Content:

### 0. Objectives

### 1. Loading Data

### 2. Data Cleaning & Preprocessing

### 3. Descriptive Analysis (Patterns)

### 4. Feature Engineering for Prediction

### 5. Baseline Forecasting Models

### 6. ML Models (Tree-based + Neural Nets)

### 7. Wrap-up & Presentation


## 0. Objectives

Descriptive:

- Identify daily, weekly, and seasonal patterns of PM2.5/PM10.

- Compare air quality across weekdays vs weekends and daytime vs nighttime.

- Visualize pollution cycles that align with traffic, heating, or special events (e.g., fireworks, holidays).

Predictive:

- Build models to predict next-day PM2.5 levels.

- Compare classical and machine learning approaches:

- Baseline: ARIMA or Holt-Winters

- ML: Random Forest, XGBoost

- Deep Learning: LSTM / GRU


### Setup:


In [5]:
# --- Core data stack ---
import numpy as np
import pandas as pd

# --- Visualization ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Stats & time-series helpers ---
from scipy import stats                     # t-tests, basic stats
from statsmodels.tsa.seasonal import seasonal_decompose  # trend/seasonality

# --- Quality of life ---
from tqdm import tqdm  # progress bars for loops (e.g., many days/years)

## Load and Show the data!


In [7]:
# Loading PurpleAir Data Frames
df_purpleAir_44919 = pd.read_csv('44919 2016-08-26 2025-08-26 60-Minute Average.csv')
df_purpleAir_92387 = pd.read_csv('92387 2016-08-26 2025-08-26 60-Minute Average.csv')
df_purpleAir_217883 = pd.read_csv('217883 2016-08-26 2025-08-26 60-Minute Average.csv')

df_purpleAir_44919.head()

,time_stamp,humidity,temperature,pressure,voc,analog_input,pm2.5_alt|pm2.5_alt = C * (0.00030418*N1 0.0018512*N2 0.02069706*N3),deciviews,visual_range,0.3_um_count,...,1.0_um_count,2.5_um_count,5.0_um_count,10.0_um_count,pm1.0_cf_1,pm1.0_atm,pm2.5_atm,pm2.5_cf_1,pm10.0_atm,pm10.0_cf_1
0,2025-01-15T07:00:00+06:00,16,72,935.98,NaN,0.04,10.6,16.9,71.6,2470.2,...,112.26,17.30,4.834,2.622,12.0,12.0,18.4,18.4,22.2,22.2
1,2025-01-15T08:00:00+06:00,18,56,936.21,NaN,0.04,15.7,20.3,51.4,3657.7,...,164.92,26.63,6.509,3.412,20.3,19.2,29.3,30.9,36.0,36.5
2,2025-01-15T09:00:00+06:00,22,46,936.45,NaN,0.05,16.4,20.0,52.8,3550.6,...,188.31,33.56,7.445,3.638,20.6,20.3,32.9,33.9,40.7,40.7
3,2025-01-15T10:00:00+06:00,20,50,936.39,NaN,0.05,22.9,22.4,41.6,4652.3,...,275.75,48.06,10.694,5.239,26.8,23.9,39.5,46.1,51.4,55.6
4,2025-01-15T11:00:00+06:00,17,53,936.27,NaN,0.05,14.2,19.3,56.4,3283.3,...,162.32,27.93,5.728,2.579,15.7,15.5,26.4,27.0,31.8,31.9


In [14]:
df_purpleAir_44919.describe()

,humidity,temperature,pressure,voc,analog_input,pm2.5_alt|pm2.5_alt = C * (0.00030418*N1 0.0018512*N2 0.02069706*N3),deciviews,visual_range,0.3_um_count,0.5_um_count,1.0_um_count,2.5_um_count,5.0_um_count,10.0_um_count,pm1.0_cf_1,pm1.0_atm,pm2.5_atm,pm2.5_cf_1,pm10.0_atm,pm10.0_cf_1
count,5333.000000,5333.000000,5333.000000,0.0,5332.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000,5333.000000
mean,30.636790,70.917495,929.868637,NaN,0.043638,27.151416,19.671873,68.558241,5135.471273,1444.568611,337.455586,53.441378,13.772453,6.286568,31.905625,24.289162,41.189068,55.112676,52.084699,68.182205
std,12.183037,19.654373,6.650782,NaN,0.004811,42.031110,7.621115,39.697254,6569.250133,1944.365056,584.650982,96.192155,25.958029,11.662299,67.878155,44.519029,67.760454,103.685645,83.295932,127.279819
min,7.000000,21.000000,915.200000,NaN,0.040000,1.600000,4.800000,3.500000,344.600000,96.200000,16.560000,2.050000,0.564000,0.191000,0.900000,0.900000,1.900000,1.900000,2.300000,2.300000
25%,21.000000,53.000000,924.590000,NaN,0.040000,7.200000,14.000000,36.500000,1707.200000,453.800000,74.170000,10.970000,3.246000,1.480000,8.000000,8.000000,12.000000,12.000000,15.200000,15.200000
50%,28.000000,76.000000,929.510000,NaN,0.040000,11.700000,17.700000,66.300000,2710.800000,726.100000,125.220000,20.140000,5.574000,2.538000,13.600000,13.400000,20.300000,20.500000,25.300000,25.400000
75%,39.000000,87.000000,934.590000,NaN,0.050000,26.500000,23.700000,95.800000,5381.000000,1502.100000,320.970000,50.860000,11.934000,5.557000,30.800000,26.200000,42.500000,51.800000,55.600000,61.900000
max,63.000000,105.000000,958.920000,NaN,0.050000,502.700000,47.100000,240.700000,61421.400000,19760.800000,7941.620000,1514.770000,431.094000,189.362000,2331.700000,1554.700000,1557.200000,2334.500000,1559.600000,2337.100000


In [15]:
df_purpleAir_92387.describe()

,humidity,temperature,pressure,voc,analog_input,pm2.5_alt|pm2.5_alt = C * (0.00030418*N1 0.0018512*N2 0.02069706*N3),deciviews,visual_range,0.3_um_count,0.5_um_count,1.0_um_count,2.5_um_count,5.0_um_count,10.0_um_count,pm1.0_cf_1,pm1.0_atm,pm2.5_atm,pm2.5_cf_1,pm10.0_atm,pm10.0_cf_1
count,39957.000000,39957.000000,39957.000000,0.0,3.995700e+04,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000,39957.000000
mean,19.943940,81.008960,926.661536,NaN,2.000000e-02,15.661348,15.775579,100.082421,3240.631486,870.619231,168.493145,13.897153,3.178610,0.832086,17.449303,14.140586,20.990545,26.070318,23.727107,28.318770
std,5.090616,4.515667,6.467675,NaN,1.093236e-14,26.514077,7.379462,58.532837,4542.470653,1305.992256,323.576579,30.544342,6.758318,1.868609,27.876624,18.371949,30.082495,45.458339,34.090651,50.234441
min,6.000000,62.000000,861.270000,NaN,2.000000e-02,0.100000,0.500000,3.700000,29.600000,7.800000,1.170000,0.050000,0.021000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,78.000000,921.830000,NaN,2.000000e-02,4.100000,10.800000,54.700000,1081.200000,261.100000,35.680000,1.940000,0.522000,0.178000,4.400000,4.400000,6.000000,6.000000,6.400000,6.400000
50%,20.000000,81.000000,926.290000,NaN,2.000000e-02,7.000000,14.000000,96.500000,1689.300000,430.200000,64.790000,3.760000,1.018000,0.328000,8.100000,8.100000,10.900000,10.900000,11.600000,11.600000
75%,23.000000,84.000000,931.050000,NaN,2.000000e-02,15.800000,19.600000,132.400000,3407.600000,910.000000,163.660000,12.100000,2.800000,0.728000,18.300000,18.100000,26.100000,26.300000,28.300000,28.300000
max,42.000000,97.000000,952.720000,NaN,2.000000e-02,450.200000,46.500000,370.300000,57487.700000,17631.200000,6290.010000,638.830000,140.310000,41.088000,370.800000,246.400000,489.000000,734.500000,558.100000,838.300000


In [16]:
df_purpleAir_217883.describe()

,humidity,temperature,pressure,voc,analog_input,pm2.5_alt|pm2.5_alt = C * (0.00030418*N1 0.0018512*N2 0.02069706*N3),deciviews,visual_range,0.3_um_count,0.5_um_count,1.0_um_count,2.5_um_count,5.0_um_count,10.0_um_count,pm1.0_cf_1,pm1.0_atm,pm2.5_atm,pm2.5_cf_1,pm10.0_atm,pm10.0_cf_1
count,4898.000000,4898.000000,4898.000000,0.0,4898.0,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,19.289098,83.481421,924.192999,NaN,0.0,11.858983,13.618416,113.772397,2133.342956,643.003348,133.679198,10.160135,2.480249,0.584667,11.979053,10.533463,16.666313,19.346325,18.679339,20.854614
std,4.547767,1.885903,5.716168,NaN,0.0,16.244929,5.746021,48.688419,2533.300309,782.053027,203.377125,18.687590,4.493325,0.895643,15.650932,10.715740,18.726916,27.714401,21.665030,30.389882
min,6.000000,76.000000,910.760000,NaN,0.0,1.400000,4.400000,4.400000,306.400000,90.400000,12.470000,0.560000,0.150000,0.031000,0.900000,0.900000,1.400000,1.400000,1.500000,1.500000
25%,16.000000,82.000000,919.630000,NaN,0.0,4.400000,9.700000,80.025000,904.500000,267.625000,44.355000,2.330000,0.658000,0.170000,4.500000,4.500000,6.600000,6.600000,7.000000,7.000000
50%,19.000000,83.000000,923.850000,NaN,0.0,6.500000,12.000000,117.200000,1292.550000,383.500000,67.265000,3.800000,1.087500,0.302000,6.800000,6.800000,10.050000,10.050000,10.800000,10.800000
75%,22.000000,85.000000,928.630000,NaN,0.0,11.500000,15.800000,148.375000,2151.625000,643.000000,126.525000,8.700000,2.265500,0.604000,12.000000,12.000000,18.600000,18.775000,20.000000,20.100000
max,34.000000,90.000000,940.780000,NaN,0.0,395.200000,44.900000,251.400000,49061.200000,15670.400000,5436.610000,478.410000,118.250000,19.984000,321.700000,213.700000,437.100000,656.900000,487.200000,732.000000


In [17]:
df_purpleAir_44919.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5333 entries, 0 to 5332
Data columns (total 21 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   time_stamp                                                                5333 non-null   object 
 1   humidity                                                                  5333 non-null   int64  
 2   temperature                                                               5333 non-null   int64  
 3   pressure                                                                  5333 non-null   float64
 4   voc                                                                       0 non-null      float64
 5   analog_input                                                              5332 non-null   float64
 6   pm2.5_alt|pm2.5_alt = C * (0.00030418*N1   0.0018512*N2   0.0206

In [18]:
df_purpleAir_92387.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39957 entries, 0 to 39956
Data columns (total 21 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   time_stamp                                                                39957 non-null  object 
 1   humidity                                                                  39957 non-null  int64  
 2   temperature                                                               39957 non-null  int64  
 3   pressure                                                                  39957 non-null  float64
 4   voc                                                                       0 non-null      float64
 5   analog_input                                                              39957 non-null  float64
 6   pm2.5_alt|pm2.5_alt = C * (0.00030418*N1   0.0018512*N2   0.02

In [19]:
df_purpleAir_217883.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 21 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   time_stamp                                                                4898 non-null   object 
 1   humidity                                                                  4898 non-null   int64  
 2   temperature                                                               4898 non-null   int64  
 3   pressure                                                                  4898 non-null   float64
 4   voc                                                                       0 non-null      float64
 5   analog_input                                                              4898 non-null   int64  
 6   pm2.5_alt|pm2.5_alt = C * (0.00030418*N1   0.0018512*N2   0.0206

## Define PurpleAir DF features:


### `humidity`

- **What:** Relative humidity.
- **Units:** % (0–100).
- **Notes:** Can be biased on some devices; consider smoothing and bounds checks.

---

### `temperature`

- **What:** Ambient temperature reported by the onboard environmental sensor.
- **Units:** °F (PurpleAir commonly reports Fahrenheit via API).
- **Notes:** Convert to °C if needed: `(F − 32) × 5/9`. Can read slightly high in sun or near walls.

---

### `pressure`

- **What:** Barometric pressure at the sensor.
- **Units:** hPa (hectopascals).
- **Notes:** Varies with elevation and weather; Bishkek (\~760 m) typically shows \~920–940 hPa.

---

### `voc`

- **What:** Volatile Organic Compounds signal or index (only on models with a VOC sensor).
- **Units:** Device/firmware dependent (often an index, sometimes absent).
- **Notes:** Many PurpleAir units do not report VOC → expect `NaN` frequently.

---

### `analog_input`

- **What:** Raw analog input channel for external integrations.
- **Units:** Unitless (normalized 0–1) or device-specific voltage proxy.
- **Notes:** Often unused (constant near `0.02–0.05` or `0.0`). Treat as optional/diagnostic.

---

### `pm2.5_alt`

- **What:** Alternate PM2.5 mass concentration derived from particle counts (the Lance Wallace method).
- **Units:** µg/m³.
- **Formula:**

  ```text
  pm2.5_alt = C * (0.00030418*N1 + 0.0018512*N2 + 0.02069706*N3)
  ```

  where **C** is a constant “CF factor” (commonly 3.0; some use 3.4) and **N1–N3** are small-particle count bins from the sensor.

- **Notes:** Useful when comparing against `cf_1` and `atm` outputs; sensitive to count noise.

---

### `deciviews`

- **What:** Logarithmic haze metric (higher = hazier).
- **Units:** deciview (dv).
- **Notes:** Derived from estimated light extinction; intended for visibility/haze interpretation rather than compliance reporting.

---

### `visual_range`

- **What:** Estimated visibility distance derived from aerosol extinction.
- **Units:** Typically km.
- **Notes:** Lower values indicate poorer visibility; derived (not directly measured).

---

### `0.3_um_count`, `0.5_um_count`, `1.0_um_count`, `2.5_um_count`, `5.0_um_count`, `10.0_um_count`

- **What:** Particle number counts in size bins ≥0.3 µm, ≥0.5 µm, …, ≥10 µm.
- **Units:** Particles per 0.1 L (Plantower convention) or device-specific count units.
- **Notes:** Extremely wide dynamic range—plot on **log scale**. Great for research/QC and diagnosing sensor issues.

---

### `pm1.0_cf_1`, `pm2.5_cf_1`, `pm10.0_cf_1`

- **What:** Mass concentrations computed by the sensor’s **CF=1** factory algorithm.
- **Units:** µg/m³.
- **Use:** Good for method comparison and diagnostics; may not best match outdoor regulatory references.

---

### `pm1.0_atm`, `pm2.5_atm`, `pm10.0_atm`

- **What:** Mass concentrations with the sensor’s **“atmospheric”** correction.
- **Units:** µg/m³.
- **Use (recommended):** For outdoor reporting and AQI conversion, use `*_atm` (especially `pm2.5_atm`).
- **Notes:** Often aligns better with ambient aerosol than `cf_1`.


In [12]:
# Loading NOAA Weather DF:
df_weather = pd.read_csv('isd_lite_383530_99999_2016_2025.csv')
df_weather.head()

,datetime_utc,datetime_local,air_temp_c,dewpoint_c,slp_hpa,wind_dir_deg,wind_speed_ms,sky_cover_code,precip_1h_mm,precip_6h_mm,year,month,day,hour
0,2016-01-01 00:00:00+00:00,2016-01-01 06:00:00+06:00,4.9,-5.8,1023.4,280,1.0,8,NaN,NaN,2016,1,1,0
1,2016-01-01 03:00:00+00:00,2016-01-01 09:00:00+06:00,4.7,-3.7,1026.5,140,1.0,8,NaN,NaN,2016,1,1,3
2,2016-01-01 06:00:00+00:00,2016-01-01 12:00:00+06:00,11.6,-2.2,1024.7,140,2.0,5,NaN,NaN,2016,1,1,6
3,2016-01-01 09:00:00+00:00,2016-01-01 15:00:00+06:00,13.8,-2.2,1023.2,50,2.0,5,NaN,NaN,2016,1,1,9
4,2016-01-01 12:00:00+00:00,2016-01-01 18:00:00+06:00,9.5,-3.3,1024.6,90,1.0,5,NaN,NaN,2016,1,1,12


In [13]:
df_weather.describe()

,air_temp_c,dewpoint_c,slp_hpa,wind_dir_deg,wind_speed_ms,sky_cover_code,precip_1h_mm,precip_6h_mm,year,month,day,hour
count,26495.000000,26465.000000,26304.000000,26532.000000,22147.000000,26532.000000,0.0,3.000000,26532.000000,26532.000000,26532.000000,26532.000000
mean,12.906243,2.589972,1017.827783,161.336876,1.472841,-1398.028155,NaN,3.666667,2020.357154,6.405548,15.700814,10.526082
std,11.714525,6.898117,10.446068,231.462566,1.942067,3473.869635,NaN,3.511885,2.797270,3.414833,8.790157,6.902611
min,-26.600000,-29.000000,994.500000,-9999.000000,0.000000,-9999.000000,NaN,0.000000,2016.000000,1.000000,1.000000,0.000000
25%,3.500000,-2.300000,1009.575000,40.000000,1.000000,1.000000,NaN,2.000000,2018.000000,3.000000,8.000000,6.000000
50%,13.600000,3.400000,1017.100000,180.000000,1.000000,5.000000,NaN,4.000000,2020.000000,6.000000,16.000000,9.000000
75%,22.200000,8.100000,1025.400000,270.000000,2.000000,8.000000,NaN,5.500000,2023.000000,9.000000,23.000000,18.000000
max,39.600000,20.700000,1058.900000,360.000000,53.000000,9.000000,NaN,7.000000,2025.000000,12.000000,31.000000,21.000000


In [20]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26532 entries, 0 to 26531
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datetime_utc    26532 non-null  object 
 1   datetime_local  26532 non-null  object 
 2   air_temp_c      26495 non-null  float64
 3   dewpoint_c      26465 non-null  float64
 4   slp_hpa         26304 non-null  float64
 5   wind_dir_deg    26532 non-null  int64  
 6   wind_speed_ms   22147 non-null  float64
 7   sky_cover_code  26532 non-null  int64  
 8   precip_1h_mm    0 non-null      float64
 9   precip_6h_mm    3 non-null      float64
 10  year            26532 non-null  int64  
 11  month           26532 non-null  int64  
 12  day             26532 non-null  int64  
 13  hour            26532 non-null  int64  
dtypes: float64(6), int64(6), object(2)
memory usage: 2.8+ MB


## Define NOAA Weather DB features:


### `air_temp_c`

- **What:** Dry-bulb air temperature (near-surface).
- **Units:** °C (converted from tenths of °C in raw file).
- **Typical range:** −50 to 60 °C.
- **Missing code in raw:** `-9999` → set to `NaN`.

---

### `dewpoint_c`

- **What:** Dew point temperature (moisture content proxy).
- **Units:** °C (from tenths of °C).
- **Typical range:** −60 to 35 °C.
- **Missing:** `-9999`.

---

### `slp_hpa`

- **What:** Sea-level pressure.
- **Units:** hPa (from tenths of hPa).
- **Typical range:** 870–1085 hPa.
- **Missing:** `-9999`.

---

### `wind_dir_deg`

- **What:** Direction **from** which the wind blows.
- **Units:** degrees (0–360; 0/360 = north).
- **Conventions:** Calm winds can have speed 0 and any direction; some feeds use special codes for “variable.”
- **Missing:** often `-9999` in ISD-Lite → treat as `NaN`.

---

### `wind_speed_ms`

- **What:** Wind speed.
- **Units:** m/s (from tenths of m/s).
- **Typical range:** 0–60+ m/s (extremes possible in storms).
- **Missing:** `-9999`.

---

### `sky_cover_code`

- **What:** Total cloud cover (coded).
- **Codes (rule-of-thumb):**

  - `0–8` ≈ oktas (0 = clear, 8 = overcast)
  - `9` = sky obscured/indeterminable
  - **Missing:** `-9999`

- **Note:** Use as **categorical**; don’t average raw codes.

---

### `precip_1h_mm`

- **What:** Precipitation in the last 1 hour.
- **Units:** mm (from tenths of mm).
- **Missing:** `-9999`. Zeros are valid (no rain).

---

### `precip_6h_mm`

- **What:** Precipitation in the last 6 hours.
- **Units:** mm (from tenths of mm).
- **Missing:** `-9999`.

---

### `year`, `month`, `day`, `hour`

- **What:** Timestamp components (ISD/ISD-Lite are in **UTC**).
- **Hour range:** 0–23 (if you see max 21, you may have filtered or resampled).
